
#H2O AutoML Regression Demo
##This is a Jupyter Notebook. When you execute code within the notebook, the results appear beneath the code. To execute a code chunk, place your cursor on the cell and press Shift+Enter.

#Start H2O
##Import the h2o Python module and H2OAutoML class and initialize a local H2O cluster.

In [7]:
!pip install h2o
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,14 mins 14 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.2
H2O_cluster_version_age:,13 days
H2O_cluster_name:,H2O_from_python_unknownUser_q2k8d0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.179 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"



#Load Data
##For the AutoML regression demo, we use the Combined Cycle Power Plant dataset. The goal here is to predict the energy output (in megawatts), given the temperature, ambient pressure, relative humidity and exhaust vacuum values. In this demo, you will use H2O's AutoML to outperform the state of the art results on this task.

In [8]:
# Use local data file or download from GitHub
import os
docker_data_path = "/home/h2o/data/automl/powerplant_output.csv"
if os.path.isfile(docker_data_path):
  data_path = docker_data_path
else:
  data_path = "https://github.com/h2oai/h2o-tutorials/raw/master/h2o-world-2017/automl/data/powerplant_output.csv"


# Load data into H2O
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


Next, let's identify the response column and save the column name as y. In this dataset, we will use all columns except the response as predictors, so we can skip setting the x argument explicitly.




In [9]:
y = "HourlyEnergyOutputMW"


Lastly, let's split the data into two frames, a train (80%) and a test frame (20%). The test frame will be used to score the leaderboard and to demonstrate how to generate predictions using an AutoML leader model.



In [10]:

splits = df.split_frame(ratios = [0.8], seed = 1)
train = splits[0]
test = splits[1]

#Run AutoML
##Run AutoML, stopping after 60 seconds. The max_runtime_secs argument provides a way to limit the AutoML run by time. When using a time-limited stopping criterion, the number of models train will vary between runs. If different hardware is used or even if the same machine is used but the available compute resources on that machine are not the same between runs, then AutoML may be able to train more models on one run vs another.

##The test frame is passed explicitly to the leaderboard_frame argument here, which means that instead of using cross-validated metrics, we use test set metrics for generating the leaderboard.

In [11]:
aml = H2OAutoML(max_runtime_secs = 60, seed = 1, project_name = "powerplant_lb_frame")
aml.train(y = y, training_frame = train, leaderboard_frame = test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


##For demonstration purposes, we will also execute a second AutoML run, this time providing the original, full dataset, df (without passing a leaderboard_frame). This is a more efficient use of our data since we can use 100% of the data for training, rather than 80% like we did above. This time our leaderboard will use cross-validated metrics.

##Note: Using an explicit leaderboard_frame for scoring may be useful in some cases, which is why the option is available.

In [12]:

aml2 = H2OAutoML(max_runtime_secs = 60, seed = 1, project_name = "powerplant_full_data")
aml2.train(y = y, training_frame = df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


#Leaderboard
##Next, we will view the AutoML Leaderboard. Since we specified a leaderboard_frame in the H2OAutoML.train() method for scoring and ranking the models, the AutoML leaderboard uses the performance on this data to rank the models.

#After viewing the "powerplant_lb_frame" AutoML project leaderboard, we compare that to the leaderboard for the "powerplant_full_data" project. We can see that the results are better when the full dataset is used for training.

#A default performance metric for each machine learning task (binary classification, multiclass classification, regression) is specified internally and the leaderboard will be sorted by that metric. In the case of regression, the default ranking metric is mean residual deviance. In the future, the user will be able to specify any of the H2O metrics so that different metrics can be used to generate rankings on the leaderboard.

In [13]:
aml.leaderboard.head()


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_grid__1_AutoML_20201130_175338_model_1,11.7239,3.42402,11.7239,2.34031,0.00751901
XGBoost_grid__1_AutoML_20201130_175338_model_2,14.4416,3.80021,14.4416,2.76009,0.0083476
XGBoost_3_AutoML_20201130_175338,15.8135,3.97662,15.8135,2.99246,0.00875027
GBM_grid__1_AutoML_20201130_175338_model_1,16.0713,4.00891,16.0713,2.98583,0.00881389
XGBoost_2_AutoML_20201130_175338,20.6318,4.54223,20.6318,3.50621,0.00997551
DeepLearning_1_AutoML_20201130_175338,22.0552,4.6963,22.0552,3.62343,0.0102724
XRT_1_AutoML_20201130_175338,22.7843,4.77329,22.7843,3.35047,0.0104907
DRF_1_AutoML_20201130_175338,25.0549,5.00549,25.0549,3.46702,0.010972
GLM_1_AutoML_20201130_175338,25.4781,5.04759,25.4781,4.01351,0.0110303
DeepLearning_grid__1_AutoML_20201130_175338_model_1,26.6211,5.15956,26.6211,3.93697,0.011377


##Now we will view a snapshot of the top models. Here we should see the two Stacked Ensembles at or near the top of the leaderboard. Stacked Ensembles can almost always outperform a single model


##This dataset comes from the UCI Machine Learning Repository of machine learning datasets. The data was used in a publication in the International Journal of Electrical Power & Energy Systems in 2014. In the paper, the authors achieved a mean absolute error (MAE) of 2.818 and a Root Mean-Squared Error (RMSE) of 3.787 on their best model. So, with H2O's AutoML, we've already beaten the state-of-the-art in just 60 seconds of compute time!

#Predict Using Leader Model
##If you need to generate predictions on a test set, you can make predictions on the "H2OAutoML" object directly, or on the leader model object.



In [14]:

pred = aml.predict(test)
pred.head()

xgboost prediction progress: |████████████████████████████████████████████| 100%


predict
485.695
473.756
467.413
450.41
448.128
468.133
442.936
466.101
442.655
431.161



##If needed, the standard model_performance() method can be applied to the AutoML leader model and a test set to generate an H2O model performance object.

In [15]:

perf = aml.leader.model_performance(test)
perf


ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 11.723888817434862
RMSE: 3.4240164744689623
MAE: 2.3403112773602754
RMSLE: 0.007519009129557612
Mean Residual Deviance: 11.723888817434862
